# <center>**Data Augmentation**</center>

In [4]:
import os
import cv2
import numpy as np
import albumentations as A

from collections import Counter

In [5]:
# Define the paths of the train labels and the train images
train_images_path = "datasets/weapon-detection-16/train/images"
train_labels_path = "datasets/weapon-detection-16/train/labels"

In [ ]:
# Define an advanced augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),  # 50% chance to flip the image horizontally
    A.VerticalFlip(p=0.2),  # 20% chance to flip vertically
    A.Rotate(limit=30, p=0.5),  # Random rotation up to ±30 degrees
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=20, p=0.5),  # Random shift, scale, rotation
    # A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, p=0.5),  # Randomly removes small patches
    # A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.5),  # cutout is depresiated 
    A.CoarseDropout(
    min_holes=1,  # Minimum number of holes
    max_holes=8,  # Maximum number of holes
    min_height=16,  # Minimum height of holes
    max_height=32,  # Maximum height of holes
    min_width=16,  # Minimum width of holes
    max_width=32,  # Maximum width of holes
    p=0.5),
    A.RandomBrightnessContrast(p=0.5),  # Adjust brightness and contrast
    A.HueSaturationValue(p=0.5),  # Adjust hue, saturation, and value (color)
    A.GaussianBlur(p=0.3),  # Apply Gaussian blur to simulate different camera qualities
    A.Resize(640, 640),  # Resize images to 640x640 for YOLOv8 compatibility
], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))  # Ensures bounding boxes are adjusted

/opt/miniconda3/envs/pytorch/lib/python3.12/site-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/var/folders/xs/x0j0zz5n3cx754v1hbtylgkm0000gn/T/ipykernel_9829/1780926763.py:8: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.5),  # cutout is depresiated


In [7]:
# Function to read YOLO labels
def read_yolo_labels(label_path):
    with open(label_path, "r") as f:
        labels = [line.strip().split() for line in f.readlines()]
    return labels

In [8]:
# Function to write YOLO labels
def write_yolo_labels(label_path, labels):
    with open(label_path, "w") as f:
        for label in labels:
            f.write(" ".join(map(str, label)) + "\n")

In [9]:
# Get all images and labels
image_files = [f for f in os.listdir(train_images_path) if f.endswith(".jpg") or f.endswith(".png")]

In [10]:
# Augment only images that contain ONLY pistols (Class 0)
augmentation_count = 4  # Number of times to augment each pistol-only image

In [11]:
for img_file in image_files:
    label_file = img_file.replace(".jpg", ".txt").replace(".png", ".txt")
    label_path = os.path.join(train_labels_path, label_file)

    if not os.path.exists(label_path):
        continue

    labels = read_yolo_labels(label_path)

    # Check if the image contains only pistols (Class 0) and no heavy guns (Class 1)
    class_ids = {int(line[0]) for line in labels}  # Extract unique class IDs from labels
    if class_ids != {0}:  # If there's anything other than class 0, skip the image
        continue

    img_path = os.path.join(train_images_path, img_file)
    img = cv2.imread(img_path)
    height, width, _ = img.shape

    # Convert YOLO bbox to pixel coordinates
    bbox_list = []
    class_labels = []
    for line in labels:
        class_id, x, y, w, h = map(float, line)
        bbox_list.append([x, y, w, h])
        class_labels.append(int(class_id))

    # Generate multiple augmented images per pistol-only image
    for i in range(augmentation_count):
        augmented = transform(image=img, bboxes=bbox_list, class_labels=class_labels)
        augmented_img = augmented["image"]
        augmented_bboxes = augmented["bboxes"]

        # Save augmented image
        aug_img_name = f"aug_{i}_{img_file}"
        aug_img_path = os.path.join(train_images_path, aug_img_name)
        cv2.imwrite(aug_img_path, augmented_img)

        # Convert back to YOLO format and save labels
        final_labels = [[class_labels[idx]] + list(bbox) for idx, bbox in enumerate(augmented_bboxes)]
        aug_label_name = f"aug_{i}_{label_file}"
        aug_label_path = os.path.join(train_labels_path, aug_label_name)
        write_yolo_labels(aug_label_path, final_labels)

print('Data augmentation done for pistol- only images!')

Data augmentation done for pistol- only images!


In [12]:
def count_classes_in_yolo_labels(directory):
    class_counts = Counter()
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as file:
                for line in file:
                    class_id = line.split()[0]  # Extract the first value (class ID)
                    class_counts[class_id] += 1

    return class_counts

In [13]:
train_dir = "datasets/weapon-detection-16/train/labels"
train_class_counts = count_classes_in_yolo_labels(train_dir)
print("Class counts in train labels:")
print(train_class_counts)

Class counts in train labels:
Counter({'0': 12725, '1': 10659})


In [14]:
validation_dir = "datasets/weapon-detection-16/valid/labels"
validation_class_counts = count_classes_in_yolo_labels(validation_dir)
print("\nClass counts in validation labels:")
print(validation_class_counts)


Class counts in validation labels:
Counter({'1': 919, '0': 396})
